In [1]:
import pybullet as pyb
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations


In [4]:
# Set the paths to the coactivation file and the log file
coactivation_file = "/home/avbiswas/RLTCA/Learning_Analyser/trained/model/snake/caterpillar/pca_learn_ExcCoact_general_3d_9-14B/pca_euler_snakeV0_caterpillar_general_babble_0.86696607_9d.txt"
v_logs = "/home/avbiswas/RLTCA/ReducedMimic/output/velocity_policy.dat"

In [5]:
with open(coactivation_file) as f:
    # Load Coactivatiion matrix of size [k, dof]
    data = json.load(f)
    coactivation_matrix = data["Basis"]
    
    
with open(v_logs) as tl:
    # Load velocity/torque/poses data of size [samples, dof]
    vels = np.loadtxt(tl)

num_coactivations = len(coactivation_matrix)

In [6]:
def getC_Hat(coactivations):
    # Input: coactivation matrix of size [k, dof]
    # Returns: inv(C' * C) * C'
    coactivation_ = np.transpose(coactivations, [1, 0])
    coactivation_t = coactivations
    ct_c = np.matmul(coactivation_t, coactivation_)
    ct_c_inv = np.linalg.inv(ct_c)
    ct_c_inv_c = np.matmul(ct_c_inv, coactivation_t)
    return ct_c_inv_c

def getError(vector, C):
    # Input: Vector : Any Query vector (velocity/torque) of dimension (dof, ) 
    # Input: C: coactivation matrix of size [k, dof]
    
    # Output: Error of the best fit model
    # E = T - AC => E = T - inv(C'.C).C'.vector

    ct_c_inv_c = getC_Hat(C)
    A = np.matmul(ct_c_inv_c, vector)
    fit = np.matmul(A, C)
    error = vector - fit
    return np.linalg.norm(error)/np.linalg.norm(vector)

def select_coactivations(coactivation_matrix, indices=None):
    # Selects individual coactivations from coactivation_matrix according to index list
    # indices must be 0-indexed!
    
    if indices is None:
        return coactivation_matrix
    else:
        return np.take(coactivation_matrix, indices, axis=0)

In [7]:
# Check residuals for individual coactivations

for i in range(num_coactivations):
    residuals = []
    for t in vels:
        selected = select_coactivations(coactivation_matrix, [i])
        err = getError(t, selected)
        residuals.append(err)
    print("Coactivation: {}, Residual Mean: {:.6f}, STD: {:.6f}".format(i + 1, np.median(residuals), np.std(residuals)))

Coactivation: 1, Residual Mean: 0.996430, STD: 0.060690
Coactivation: 2, Residual Mean: 0.845126, STD: 0.146488
Coactivation: 3, Residual Mean: 0.992163, STD: 0.031793
Coactivation: 4, Residual Mean: 0.996041, STD: 0.008218
Coactivation: 5, Residual Mean: 0.963473, STD: 0.171450
Coactivation: 6, Residual Mean: 0.988945, STD: 0.028860
Coactivation: 7, Residual Mean: 0.995064, STD: 0.028245
Coactivation: 8, Residual Mean: 0.992918, STD: 0.117543
Coactivation: 9, Residual Mean: 0.994709, STD: 0.147327


In [8]:
# Check residual for a multiple coactivation

list_to_check = [2, 5]
selected = select_coactivations(coactivation_matrix, np.array(list_to_check) - 1)
residuals = []
for t in vels:
    residuals.append(getError(t, selected))
print(np.mean(residuals))

0.6807638794784479


In [9]:
# Check residuals for coactivations [1...k]
for i in range(1, num_coactivations + 1):
    residuals = []
    for t in vels:
        selected = select_coactivations(coactivation_matrix, np.arange(0, i))
        residuals.append(getError(t, selected))
    print("Coactivations: {}, Mean: {:.3f}, STD: {:.2f}".format(np.arange(0, i)+1, np.mean(residuals), np.std(residuals)))

Coactivations: [1], Mean: 0.972, STD: 0.06
Coactivations: [1 2], Mean: 0.795, STD: 0.17
Coactivations: [1 2 3], Mean: 0.751, STD: 0.22
Coactivations: [1 2 3 4], Mean: 0.741, STD: 0.22
Coactivations: [1 2 3 4 5], Mean: 0.558, STD: 0.29
Coactivations: [1 2 3 4 5 6], Mean: 0.518, STD: 0.30
Coactivations: [1 2 3 4 5 6 7], Mean: 0.476, STD: 0.29
Coactivations: [1 2 3 4 5 6 7 8], Mean: 0.289, STD: 0.29
Coactivations: [1 2 3 4 5 6 7 8 9], Mean: 0.054, STD: 0.07


In [10]:
# Check residuals for all combinations of 'n_combo' coactivations

n_combo = 2
combs = combinations(np.arange(0, num_coactivations), n_combo) 
mean_dict = {}
std_dict = {}
for comb in combs:
    residuals = []
    selected = select_coactivations(coactivation_matrix, comb)
    for t in vels:
        # Loop over all query vectors and store the error
        residuals.append(getError(t, selected))
        
    # Record the mean and std residuals of all the vectors
    mean_residual = np.round(np.mean(residuals), 5)
    std_residual = np.std(residuals)
    comb_one_indexed = tuple([(cmb + 1) for cmb in comb])
    mean_dict[comb_one_indexed] = mean_residual
    std_dict[comb_one_indexed] = std_residual

In [12]:
# Sort by mean residual error

sorted_items = sorted(mean_dict.items(), key=lambda x: x[1])


# Display top 15 combinations

for combo, means in sorted_items[:15]:    
    print("Coactivations : {}, Mean: {:.3f}, STD: {:.3f}".format(combo, means, std_dict[combo]))

Coactivations : (2, 5), Mean: 0.681, STD: 0.207
Coactivations : (2, 9), Mean: 0.696, STD: 0.257
Coactivations : (2, 8), Mean: 0.735, STD: 0.157
Coactivations : (5, 9), Mean: 0.776, STD: 0.183
Coactivations : (5, 8), Mean: 0.784, STD: 0.189
Coactivations : (2, 3), Mean: 0.790, STD: 0.203
Coactivations : (1, 2), Mean: 0.795, STD: 0.166
Coactivations : (2, 7), Mean: 0.806, STD: 0.133
Coactivations : (2, 6), Mean: 0.807, STD: 0.143
Coactivations : (8, 9), Mean: 0.818, STD: 0.162
Coactivations : (2, 4), Mean: 0.822, STD: 0.143
Coactivations : (1, 5), Mean: 0.842, STD: 0.189
Coactivations : (5, 7), Mean: 0.848, STD: 0.185
Coactivations : (3, 5), Mean: 0.850, STD: 0.167
Coactivations : (5, 6), Mean: 0.853, STD: 0.170
